<a href="https://colab.research.google.com/github/sangho24/sogang/blob/main/EC5320_2024_2_Week9b_NLP_v2_FOR_STUDENTS_20200572.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EC5320 NLP English version

2024.10.28.<br>

Author: Hyunjoo Yang (hyang@sogang.ac.kr)<br><br>

This notebook uses FASTAI to classify texts (Natural language processing).<br><br>

Main source of codes: <br>

https://github.com/fastai/fastbook/blob/master/10_nlp.ipynb

# 1. Install and import libraries

In [ ]:
import fastai
print(fastai.__version__)

#from fastai.vision.all import *
from fastai.text.all import *
#from fastai.collab import *
#from fastai.tabular.all import *

2.7.18


# 2. Load IMDB data

In [ ]:
path = untar_data(URLs.IMDB)

In [ ]:
path

Path('/root/.fastai/data/imdb')

In [ ]:
!ls /root/.fastai/data/imdb/train/pos

0_9.txt       11607_10.txt  1964_7.txt	 3571_7.txt   5178_10.txt  6787_9.txt	8395_8.txt
10000_8.txt   11608_10.txt  1965_7.txt	 3572_10.txt  5179_7.txt   6788_9.txt	8396_8.txt
10001_10.txt  11609_10.txt  1966_10.txt  3573_7.txt   5180_9.txt   678_8.txt	8397_10.txt
10002_7.txt   11610_10.txt  1967_8.txt	 3574_10.txt  518_10.txt   6789_8.txt	839_7.txt
10003_8.txt   116_10.txt    1968_8.txt	 3575_10.txt  5181_10.txt  6790_10.txt	8398_8.txt
10004_8.txt   11611_9.txt   1969_10.txt  3576_9.txt   5182_8.txt   679_10.txt	8399_10.txt
10005_7.txt   11612_10.txt  196_9.txt	 3577_7.txt   5183_9.txt   6791_10.txt	8400_7.txt
10006_7.txt   11613_7.txt   1970_9.txt	 3578_7.txt   5184_9.txt   6792_10.txt	8401_10.txt
10007_7.txt   11614_7.txt   1971_9.txt	 3579_8.txt   5185_10.txt  6793_10.txt	8402_10.txt
10008_7.txt   11615_9.txt   1972_10.txt  3580_10.txt  5186_10.txt  6794_10.txt	8403_10.txt
1000_8.txt    11616_8.txt   1973_8.txt	 358_10.txt   5187_7.txt   6795_9.txt	8404_10.txt
10009_9.txt   11617_9

In [ ]:
!ls /root/.fastai/data/imdb/train/neg

0_3.txt      11405_1.txt  156_1.txt   2969_3.txt  4376_1.txt  5783_4.txt  7190_4.txt  8596_3.txt
10000_4.txt  11406_1.txt  1562_1.txt  2970_4.txt  4377_3.txt  578_3.txt   7191_3.txt  8597_1.txt
10001_4.txt  11407_1.txt  1563_1.txt  2971_3.txt  4378_4.txt  5784_4.txt  7192_1.txt  8598_1.txt
10002_1.txt  11408_1.txt  1564_3.txt  2972_1.txt  4379_1.txt  5785_4.txt  7193_2.txt  8599_2.txt
10003_1.txt  11409_1.txt  1565_1.txt  2973_2.txt  4380_1.txt  5786_2.txt  7194_1.txt  8600_1.txt
10004_3.txt  11410_3.txt  1566_3.txt  2974_1.txt  4381_2.txt  5787_4.txt  719_4.txt   8601_1.txt
1000_4.txt   11411_4.txt  1567_1.txt  297_4.txt   4382_3.txt  5788_1.txt  7195_2.txt  860_1.txt
10005_3.txt  11412_1.txt  1568_1.txt  2975_4.txt  4383_1.txt  5789_1.txt  7196_4.txt  8602_2.txt
10006_4.txt  11413_1.txt  1569_2.txt  2976_1.txt  4384_1.txt  5790_1.txt  7197_2.txt  8603_2.txt
10007_1.txt  1141_3.txt   1570_2.txt  2977_1.txt  438_4.txt   5791_1.txt  7198_1.txt  8604_3.txt
10008_2.txt  11414_1.txt  1571_

In [ ]:
!ls /root/.fastai/data/imdb/test

labeledBow.feat  neg  pos


In [ ]:
# randomly remove 90% of files to speed up

for dirpath, dirnames, filenames in os.walk(path):
    txt_files = [f for f in filenames if f.endswith('.txt')]
    num_files_to_remove = int(len(txt_files) * 0.9)
    files_to_remove = random.sample(txt_files, num_files_to_remove)
    for file in files_to_remove:
        file_path = os.path.join(dirpath, file)
        os.remove(file_path)

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
txt = files[0].open().read(); txt[:200]

'A trite fish-out-of-water story about two friends from the midwest who move to the big city to seek their fortune. They become Playboy bunnies, and nothing particularly surprising happens after that. '

# 3. Classify sentiment: "positive" v.s. "negative"

In [ ]:
text_dls = TextDataLoaders.from_folder(path, valid='test')

In [ ]:
%%time

text_learn = text_classifier_learner(text_dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()
text_learn.fine_tune(5, 1e-2)

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

epoch,train_loss,valid_loss,accuracy,time
0,0.568205,0.474843,0.773200,00:09


/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.438996,0.367829,0.836400,00:14
1,0.391458,0.334555,0.850000,00:16
2,0.321652,0.320153,0.852400,00:14
3,0.245587,0.315980,0.867600,00:15
4,0.188723,0.316892,0.872000,00:15


CPU times: user 56.1 s, sys: 6.43 s, total: 1min 2s
Wall time: 1min 27s


In [ ]:
# try your own review

text_learn.predict("I really hate this movie so that I wanted to turn off my TV so fast. Why anyone bother to watch this low quality movie? Characters are also mismatched.")

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


('neg', tensor(0), tensor([0.6733, 0.3267]))

In [ ]:
text_learn.predict("It is so so so bad! too boring and makes me sleepy")

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


('neg', tensor(0), tensor([0.9778, 0.0222]))

In [ ]:
text_learn.predict("best movie I have ever seen in my life!")

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


('pos', tensor(1), tensor([1.6124e-05, 9.9998e-01]))

# 4. Understand how text data are pre-processed for learning

## 4.1 Word tokenizer

In [ ]:
# word tokenizer

spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#200) ['After','wasting','2','hours','of','my','life','watching','this','movie','on','late','night','television',',','I','went','back','and','reread','some','of','the','IMDb','material',',','to','remind','myself','of'...]


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#219) ['xxbos','xxmaj','after','wasting','2','hours','of','my','life','watching','this','movie','on','late','night','television',',','i','went','back','and','reread','some','of','the','imdb','material',',','to','remind','myself'...]


In [ ]:
# default FastAI text processing rules

defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [ ]:
# example of how setences are processed

coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

In [ ]:
txts = L(o.open().read() for o in files[:10])

## 4.2 Subword tokenizer

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [ ]:
subword(1000)


'▁A ▁tri te ▁fi sh - out - of - wa ter ▁story ▁ab out ▁t wo ▁friend s ▁fr om ▁the ▁mid we st ▁who ▁move ▁to ▁the ▁b ig ▁c ity ▁to ▁see k ▁the ir ▁fortune .'

In [ ]:
subword(200)


'▁ A ▁ t r it e ▁fi s h - out - o f - w a t er ▁s t or y ▁a b out ▁ t w o ▁ f r i e nd s ▁ f'

In [ ]:
subword(10000)

'▁A ▁tri te ▁fi sh - out - of - wa ter ▁story ▁ab out ▁t wo ▁friend s ▁fr om ▁the ▁mid we st ▁who ▁move ▁to ▁the ▁b ig ▁c ity ▁to ▁see k ▁the ir ▁fortune .'

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#62) ['xxbos','a','trite','fish','-','out','-','of','-','water','story','about','two','friends','from','the','midwest','who','move','to','the','big','city','to','seek','their','fortune','.','xxmaj','they','become'...]


In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#62) ['xxbos','a','trite','fish','-','out','-','of','-','water'...]

## 4.3 Numericalize

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

'(#152) [\'xxunk\',\'xxpad\',\'xxbos\',\'xxeos\',\'xxfld\',\'xxrep\',\'xxwrep\',\'xxup\',\'xxmaj\',\'the\',\'.\',\',\',\'and\',\'a\',\'of\',\'to\',\'in\',\'it\',\'"\',\'was\'...]'

In [ ]:
nums = num(toks)[:20]; nums

TensorText([  2,  13,   0,   0,  40,  41,  40,  14,  40,   0, 105,  68,  85,
              0, 106,   9,   0,  48,   0,  15])

In [ ]:
' '.join(num.vocab[o] for o in nums)

'xxbos a xxunk xxunk - out - of - xxunk story about two xxunk from the xxunk who xxunk to'

In [ ]:
from IPython.display import display,HTML
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [ ]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)

In [ ]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 38]), torch.Size([64, 38]))

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos a xxunk xxunk - out - of - xxunk story about two xxunk from the xxunk who xxunk to'

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

'a xxunk xxunk - out - of - xxunk story about two xxunk from the xxunk who xxunk to the'

# 5. Train more formally

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj berlin - born in 1942 xxmaj margarethe von xxmaj trotta was an actress and now she is a very important director and writer . xxmaj she has been described , perhaps even unfairly xxunk , as a director whose commitment to bringing a woman 's sensibility to the screen outweighs her artistic strengths . "" rosenstrasse , "" which has garnered mixed and even strange reviews ( the xxmaj new xxmaj york xxmaj times article was one of the most negatively aggressive reviews xxmaj i 've ever read in that paper ) is not a perfect film . xxmaj it is a fine movie and a testament to a rare xxunk of successful opposition to the xxunk xxmaj nazi regime by , of all peoples , xxunk powerless xxmaj germans demonstrating in a xxmaj berlin street . \n\n xxmaj co - writer von xxmaj trotta uses the actual",pos
1,"xxbos xxmaj i 've rented and watched this movie for the 1st time on xxup dvd without reading any reviews about it . xxmaj so , after 15 minutes of watching xxmaj i 've noticed that something is wrong with this movie ; it 's xxup terrible ! i mean , in the trailers it looked scary and serious ! \n\n i think that xxmaj eli xxmaj roth ( mr . xxmaj director ) thought that if all the characters in this film were stupid , the movie would be funny … ( so stupid , it 's funny … ? xxup wrong ! ) xxmaj he should watch and learn from better horror - comedies such xxunk xxmaj night "" , "" the xxmaj lost xxmaj boys "" and "" the xxmaj return xxmaj of the xxmaj living xxmaj dead "" ! xxmaj those are funny ! \n\n """,neg
2,"xxbos xxmaj that word ' true ' in this film 's title got my alarm bells ringing . xxmaj they rang louder when a title card referred to xxmaj america 's xxmaj civil xxmaj war as the ' war xxmaj between the xxmaj states ' ( the xxunk preferred by die - hard southerners ) . xxmaj jesse xxmaj james -- thief , slave - holder and murderer -- is described as a quiet , gentle farm boy . \n\n xxmaj how dishonest is this movie ? xxmaj there is xxup no mention of slavery , far less of the documented fact that xxmaj jesse xxmaj james 's poor xxunk mother owned slaves before the war , and that xxmaj jesse and his brother xxmaj frank actively fought to preserve slavery . xxmaj according to this movie , all those xxmaj civil xxmaj war soldiers were really fighting to decide",neg


In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

(#10) [62,209,228,292,209,426,302,180,191,336]

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [ ]:
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.547124,0.568219,0.759600,00:09


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.479704,0.469504,0.770000,00:07


In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.389688,0.361087,0.833200,00:09


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,0.309441,0.339694,0.851600,00:11
1,0.291802,0.338951,0.846400,00:10
